<a href="https://colab.research.google.com/github/Pravalika-26/DL/blob/main/DLQuestion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

def upload_dataset():

    print("Please upload your dataset file.")
    uploaded = files.upload()

    # Check if a file has been uploaded
    if uploaded:
        file_name = list(uploaded.keys())[0]
        print(f"Dataset '{file_name}' uploaded successfully.")
        return file_name
    else:
        print("No file uploaded.")
        return None

# Call the function to upload the dataset
dataset_file = upload_dataset()


Please upload your dataset file.


Saving lyrics.csv to lyrics (1).csv
Dataset 'lyrics (1).csv' uploaded successfully.


In [ ]:
# QUESTION2
# Install necessary libraries
!pip install transformers datasets pandas --quiet
import os
os.environ["WANDB_DISABLED"] = "true"  # ✅ Disable wandb before training

# Imports
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments
)

# Load and read lyrics from CSV file
csv_file = "/content/lyrics.csv"  # Make sure this file is uploaded and present in your working directory

df = pd.read_csv(csv_file)
print(df.columns)

# Assuming the lyrics column is named "lyrics". Change this if your column has a different name
lines = df["ogLyric"].dropna().astype(str).str.strip().tolist()


lines = [line for line in lines if line]  # Remove empty lines

# Convert to Hugging Face dataset
dataset = Dataset.from_dict({"text": lines})

# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Avoid padding error
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data collator (for causal LM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    save_steps=200,
    logging_steps=20,
    overwrite_output_dir=True,
    save_total_limit=1,
    prediction_loss_only=True,
    logging_dir="./logs"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save model and tokenizer
model.save_pretrained("gpt2-lyrics-finetuned")
tokenizer.save_pretrained("gpt2-lyrics-finetuned")


Index(['ogArtist', 'songName', 'badword', 'count', 'category', 'year',
       'isCensored', 'isPresent', 'ogLyric', 'kbLyric'],
      dtype='object')


Map:   0%|          | 0/1353 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-f7b8f103effc>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
20,4.145500
40,3.928700
60,3.574400
80,3.632200
100,3.601200
120,3.445300
140,3.681100
160,3.591200
180,3.105200
200,3.230900


('gpt2-lyrics-finetuned/tokenizer_config.json',
 'gpt2-lyrics-finetuned/special_tokens_map.json',
 'gpt2-lyrics-finetuned/vocab.json',
 'gpt2-lyrics-finetuned/merges.txt',
 'gpt2-lyrics-finetuned/added_tokens.json')

In [10]:

from transformers import pipeline

# Load fine-tuned model and tokenizer
generator = pipeline("text-generation", model="gpt2-lyrics-finetuned", tokenizer="gpt2-lyrics-finetuned")

# Get user prompt
prompt = input("Enter your song prompt: ")

# Generate lyrics
result = generator(prompt, max_new_tokens=50)

# Print the generated text
print("\n Generated Lyrics:")
print(result[0]['generated_text'])



Device set to use cpu


Enter your song prompt: Bought matching diamonds for six of my bitches

 Generated Lyrics:
Bought matching diamonds for six of my bitches, right? (Yeah)Yeah, I've been wearing these beautiful, gold-rimmed glasses all summer long (Whoa) (Yeah) (Yeah) (Yeah) (Yeah) (Yeah) I know, I know, I know
